# Support Vector Machine (SVM)

In [ ]:
import pandas as pd
import string
import re
import spacy
from unidecode import unidecode

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC


### Função para pré-processamento:
- Transformam todos os caracteres em minusculas;
- Remove as acentuações
- Remove as pontuações
- Remove números

In [ ]:
def cleaner(texto):
        texto = unidecode(texto)
        texto = texto.lower()
        texto = texto.translate(str.maketrans('','', string.punctuation))
        texto = re.sub(r'[0-9]+', '', texto)
        return texto

In [ ]:
df = pd.read_json('../model/News_Category_Dataset_v3.json', lines=True)
df.head()

### Preparando a tabela para o treinamento:
- Junção das tabelas 'headline' e 'short_cription'
- Remoçao das tabelas 'link', 'authors', 'date', 'headline', 'short_description'
- Com a função cleaner limpamos a tabela

In [ ]:
df.insert(loc=0, column= 'description', value= df['headline'] + df['short_description'])
df = df.drop(columns=['link', 'authors', 'date', 'headline', 'short_description'])
df['description'] = df['description'].apply(cleaner)
df.head()

### Remoção de Stop Words

In [ ]:
nlp = spacy.load("en_core_web_sm")
words = nlp.Defaults.stop_words

def stopWords(text):
    text = [word for word in text.split() if word not in list(words)]
    return ' '.join(text)
print(words)
df['description'] = df['description'].apply(stopWords)
df.head()  

### Contagem das palavras

In [ ]:
vector = CountVectorizer()
matriz = vector.fit_transform(df['description'])
contagem = pd.DataFrame(matriz)
df.insert(loc=2, column= 'counter', value= contagem)
df.head() 

In [ ]:
df['counter'] = df['description'].apply(lambda text: nlp(text).vector)
df.head()

### Algoritmo de treinamento